In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as pch
import matplotlib.backends.backend_pdf as pdf

import eleanor_constants as EL
import A_model_diagnostics as A
import B_model_foraging_task as B
import C_model_toxin_avoidance_task as C

matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
matplotlib.rcParams['svg.fonttype'] = 'path'

savename = "./figures/5.pdf"

In [ ]:
n_trials = 1000

anosmic_fed = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_fed.csv'
orthokinesis_fed = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_fed.csv'
klinokinesis_fed = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_fed.csv'
chemo_1_fed = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_1_fed.csv'

anosmic_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_averse_fed.csv'
orthokinesis_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_averse_fed.csv'
klinokinesis_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_averse_fed.csv'
chemo_1_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_averse_1_fed.csv'

In [ ]:
df = pd.read_csv("./data/trajectories/summary/modeling_Starved_acclimate_all_animals.csv")
anglelist = df["angle_delta"].dropna().values
speedlist = df["speed_mm_s"].dropna().values
speedlist = [x/2 for x in speedlist] # Convert from mm/s to mm/frame
width = 60
r = width/2
radius_food = A.get_radius_food(width)

In [ ]:
t, xO, yO = B.orthokinesis(speedlist, anglelist, width, radius_food, r, r, (r, r*1.95), None)
t, xC, yC = B.chemotaxis(speedlist, anglelist, width, radius_food, r, r, (r, r*1.95), 1)
t, xK, yK = B.klinokinesis(speedlist, anglelist, width, radius_food, r, r, (r, r*1.95), None)
t, xA, yA = B.anosmic(speedlist, anglelist, width, radius_food, r, r, (r, r*1.95), None)

t, xOA, yOA = C.orthokinesis_averse(speedlist, anglelist, width, radius_food, r, r, (r, r), None)
t, xCA, yCA = C.chemotaxis_averse(speedlist, anglelist, width, radius_food, r, r, (r, r), 1)
t, xKA, yKA = C.klinokinesis_averse(speedlist, anglelist, width, radius_food, r, r, (r, r), None)
t, xAA, yAA = C.anosmic_averse(speedlist, anglelist, width, radius_food, r, r, (r, r), None)

In [ ]:
fig = plt.figure(figsize=(7, 12))

grid = matplotlib.gridspec.GridSpec(8, 4)

ax1 = fig.add_subplot(grid[1:3, :], facecolor="None")
ax2 = fig.add_subplot(grid[5:7, :], facecolor="None")
ax3 = fig.add_subplot(grid[3, :], facecolor="None")

axA = fig.add_subplot(grid[0, 0], aspect="equal", facecolor="None")
axB = fig.add_subplot(grid[0, 1], aspect="equal", facecolor="None")
axC = fig.add_subplot(grid[0, 2], aspect="equal", facecolor="None")
axD = fig.add_subplot(grid[0, 3], aspect="equal", facecolor="None")

axAv = fig.add_subplot(grid[4, 0], aspect="equal", facecolor="None")
axBv = fig.add_subplot(grid[4, 1], aspect="equal", facecolor="None")
axCv = fig.add_subplot(grid[4, 2], aspect="equal", facecolor="None")
axDv = fig.add_subplot(grid[4, 3], aspect="equal", facecolor="None")

x_data = [xO, xC, xK, xA, xOA, xCA, xKA, xAA]
y_data = [yO, yC, yK, yA, yOA, yCA, yKA, yAA]
colors = [EL.c_ortho, EL.c_chemo, EL.c_klino, EL.c_anosmic]*2
labels = ["Orthokinesis", "Chemotaxis", "Klinokinesis", "Anosmic"]*2
fifty = 12.241810915582668
for i, ax in enumerate([axA, axB, axC, axD, axAv, axBv, axCv, axDv]):
    ax.set_xlim(0, width)
    ax.set_ylim(0, width)
    ax.axis("off")
    ax.plot(x_data[i], y_data[i], color=colors[i], zorder=20, label=labels[i])
    
    if i < 4:
        cmap = EL.cmap_food
    else:
        cmap = EL.cmap_poison
        
    for rr in np.linspace(r, r*2, 100)[::-1]:
        conc = A.get_concentration(rr, r, r, radius_food, r, r)
        color = cmap(conc/100)
        ax.add_patch(pch.Circle((r, r), rr-r, facecolor=color, edgecolor=None, zorder=1))
        
    ax.add_patch(pch.Circle((r, r), radius_food, facecolor=color, zorder=25, alpha=0.75))
    ax.add_patch(pch.Circle((r, r), radius_food, facecolor="None", edgecolor='k', zorder=25, alpha=0.5))
    ax.add_patch(pch.Circle((r, r), r, facecolor="None", edgecolor='k', clip_on=False, zorder=25, alpha=0.5))
    ax.add_patch(pch.Circle((r, r), radius_food+fifty, facecolor="None", linestyle="dotted",
                            edgecolor='k', clip_on=False, zorder=25, alpha=0.5))
    ax.scatter(x_data[i][0], y_data[i][0], marker="o", s=EL.s+5, color="k", 
               lw=0, edgecolor="None", alpha=0.75, zorder=30, clip_on=False)

handles, labels = [], []
for ax in [axA, axB, axC, axD]:
    h1, l1 = ax.get_legend_handles_labels()
    handles += h1
    labels += l1
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
legend = ax3.legend(handles, labels, frameon=False, loc='center', ncol=4, columnspacing=1,
                handletextpad=0.75, bbox_to_anchor=(0, 0, 1, 1.25), markerscale=2)
a=np.outer(np.ones(10),np.arange(0,1,0.0001))

img = ax3.imshow(a, aspect='auto', cmap=EL.cmap_food, origin="lower")
img.set_visible(False)
cax = fig.add_axes([0.075, 0.5475, 0.4, 0.0075]) 
cbar = plt.colorbar(img, cax=cax, orientation="horizontal", ticks=[0, 0.2, 0.4, 0.6, 0.8, 0.995])
cbar.ax.set_xlabel("Food Concentration (\%)")
cbar.ax.set_xticklabels(np.arange(0, 101, 20))

img = ax3.imshow(a, aspect='auto', cmap=EL.cmap_poison, origin="lower")
img.set_visible(False)
cax = fig.add_axes([0.525, 0.5475, 0.4, 0.0075]) 
cbar = plt.colorbar(img, cax=cax, orientation="horizontal", ticks=[0, 0.2, 0.4, 0.6, 0.8, 0.995])
cbar.ax.set_xlabel("Repellent Concentration (\%)")
cbar.ax.set_xticklabels(np.arange(0, 101, 20))

ax3.axis("off")

food = [orthokinesis_fed, chemo_1_fed, klinokinesis_fed, anosmic_fed]
poison = [orthokinesis_averse_fed, chemo_1_averse_fed, klinokinesis_averse_fed, anosmic_averse_fed]
for i, f in enumerate(food+poison):
    df = pd.read_csv(f, index_col=None)
    widths = df.columns.astype(int).tolist()
    widths = [int(x) for x in widths]
    means = df.mean(axis=0).tolist()
    sterr = df.sem(axis=0).tolist()
    if i < 4:
        ax = ax1
        means = [m/60 for m in means]
        sterr = [s/60 for s in sterr]
    else:
        ax = ax2
    x, y, s = [w/10 for w in widths], means, sterr
    ax.scatter(x, y, clip_on=False, alpha=1, color=colors[i], marker="o", s=EL.s, label=None)
    ax.plot(x, y, clip_on=False, alpha=0.75, color=colors[i], label=None)
    ax.errorbar(x, y, yerr=s, xerr=0, fmt='|', alpha=0.25, color=colors[i], label=None)
    
ax1.set_ylabel("Discovery time (minutes)")
ax2.set_ylabel("Time spent near repellent (\%)")
ax2.set_ylim(0, 1)
ax2.set_yticks(np.arange(0, 1.1, 0.5))
ax2.set_yticks(np.arange(0, 1.1, 0.1), minor=True)
ax1.set_ylim(0, 90)
ax1.set_yticks(np.arange(0, 91, 30))
ax1.set_yticks(np.arange(0, 90, 10), minor=True)

for ax in [ax1, ax2]:
    ax.set_xlim(3, 20)
    ax.set_xticks(np.arange(4, 21, 4))
    ax.set_xticks(np.arange(3, 20, 1), minor=True)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xlabel("Arena width (cm)")
    for label, size in zip(EL.key_labels, EL.key_sizes): 
        ytop = ax.get_ylim()[1]
        ax.plot([size, size], [0, ytop], color='k', alpha=0.5, ls="dotted", clip_on=False)
        ax.text(size-0.25, ytop, label, color='k', alpha=0.5, va="top", ha="right")

# SET BOUNDARIES AND SAVE FIGURE -----------------------------------------------
plt.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)
pp = pdf.PdfPages(savename, keep_empty=False)
pp.savefig(fig)
pp.close()
plt.show()